In [ ]:
# Import packages and set numpy random seed
import numpy as np
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential

#for GPU training
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.33)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)


In [ ]:


model = Sequential(
[
    keras.Input((128,128,3)),
    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(16, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(8, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(4, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(1, activation='sigmoid')
]
)

optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()

model.summary()
model.load_weights('data_aug_off_model.h5')


In [ ]:
@tf.function
def test_step(x):
    val_preds = model(x, training=False)    
    return val_preds

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray=[gray,gray,gray]
    to_test = np.resize(gray,(1,128,128,3))
#     print(to_test)
#     print(to_test.shape)
    result = test_step(to_test)
    result = np.reshape(tf.get_static_value(result),(1,1))[0][0]
    
    # Display the resulting frame
    prediction = ''
    if result>0.5:
        prediction = 'non-drowsy'
    else:
        prediction = 'drowsy'
       
    cv2.putText(frame, str(result), (10, 160), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.putText(frame, prediction, (10, 200), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()